In [4]:
import numpy as np
import pandas as pd 
import obspy
import creep_event_picker as cep
import datetime as dt
import matplotlib.pyplot as plt
import inflect
stringify = inflect.engine()
import math
import os
import raw_to_obspy as rto
from os import listdir
from os.path import isfile, join
import importlib

In [2]:
network = 'Central_San_Andreas_Fault'
#station = ['crr','cwc3','cwn','tabc','wkr','x46','xgh','xhr','xhr1','xhr2','xhsw','xmbc','xmd','xmm','xmr','xpk','xpk1','xrsw','xsc','xsj','xsj2','xta','xva']
station = ['xmro']
#station = ['Gok','Has','Hat','Kar','Kir','Orm','pan','pas','Tas']
path = "../../Data/{p}/SAC".format(p=network)
rto.check_dir(path)
#Creepmeter_dataframe = pd.DataFrame(columns = ['Network','Creepmeter','Start Time','End Time','Sampling rate, Hz','Sampling rate, mins','Latitude','Longitude'])
#Creepmeter_dataframe.to_csv('../../../Data/creepmeter_lookup.csv')
Creepmeter_dataframe = pd.read_csv('../../Data/creepmeter_lookup.csv',index_col=0)

Import data and create .SAC files for USGS creepmeters

In [5]:
importlib.reload(rto)
for j in range(len(station)):
    #creeping, tm_diff = rto.import_USGS(station[j],network)
    creeping, tm_diff = rto.import_USGS_C46(station[j],network)

    plt.figure()
    bins = np.arange(0,61,1)
    plt.hist(tm_diff,bins=bins)
    plt.show()


    bins_hist, hist_val = np.histogram(tm_diff,bins)
    boolarr = bins_hist > 150000
    periods = hist_val[1:][boolarr] -1
    print(periods)

    if len(periods) >= 2:
        print('warning, this needs thinking about majorly!')

        a = eval('rto.{k}_to_{g}(tm_diff)'.format(k = stringify.number_to_words(periods[0]), g = stringify.number_to_words(periods[1])))
        b = eval('rto.{k}_to_{g}(tm_diff)'.format(k = stringify.number_to_words(periods[1]), g = stringify.number_to_words(periods[0])))

        creeping_A = creeping.iloc[:a]
        creeping_B = creeping.iloc[a:b]
        creeping_C = creeping.iloc[b:]
    elif len(periods) == 1: 
        creeping_C = creeping.iloc[0:]
    

    Number_of_splits = []
    Sampling_rates_variable = []
    try:
        sampling_A = round((creeping_A.Tm.iloc[2] - creeping_A.Tm.iloc[1])/dt.timedelta(minutes=1))
        Sampling_rates_variable.append(sampling_A)
        Number_of_splits.append('A')
    except (IndexError, NameError):
        sampling_A = 0

    try:
        sampling_B = round((creeping_B.Tm.iloc[2] - creeping_B.Tm.iloc[1])/dt.timedelta(minutes=1))
        Sampling_rates_variable.append(sampling_B)
        Number_of_splits.append('B')
    except (IndexError, NameError):
        sampling_B = 0

    try:
        sampling_C = round((creeping_C.Tm.iloc[2] - creeping_C.Tm.iloc[1])/dt.timedelta(minutes=1))
        Sampling_rates_variable.append(sampling_C)
        Number_of_splits.append('C')
    except (IndexError, NameError):
        sampling_C = 0

    print(Sampling_rates_variable,Number_of_splits)
    #print(creeping_A,creeping_B,creeping_C)


    for i in range(len(Number_of_splits)):
        print('interpolating creeping_{k}'.format(k=Number_of_splits[i]))
        upsampled = rto.interpolate(eval('creeping_{k}'.format(k=Number_of_splits[i])),Sampling_rates_variable[i])
        val1_okay = math.isnan(upsampled.Creep.iloc[0])
        if val1_okay == True:
            upsampled.reset_index(inplace=True)
            upsampled.drop([0],axis=0,inplace=True)
        tr = obspy.Trace(np.array(upsampled.Creep))
        st = obspy.Stream(tr)
        st[0].stats.network = network
        st[0].stats.station = station[j]
        st[0].stats.location = '00'
        st[0].stats.channel = 'slip'
        st[0].stats.starttime = upsampled.Tm.iloc[0]
        st[0].stats.delta = 60*Sampling_rates_variable[i]
        st[0].plot()
        instrument = pd.DataFrame({'Network':[st[0].stats.network],'Creepmeter_full_name':'please fill','Creepmeter_abbrv':'please fill','File_code':['{p}_{q}'.format(p=station[j],q=i)],'Start Time':[st[0].stats.starttime],\
                                   'End Time':[st[0].stats.endtime],'Sampling rate, Hz':[st[0].stats.sampling_rate],\
                                    'Sampling rate, mins':(1/st[0].stats.sampling_rate)/60,'Latitude':['please fill'],'Longitude':['please fill']})
        
        Creepmeter_dataframe = pd.concat([Creepmeter_dataframe,instrument])
        st.write(path +'/{p}_{q}.SAC'.format(p=station[j],q=i),format='SAC')
    try:
        del(creeping_A)
    except NameError:
        print('No A')
    try:
        del(creeping_B)
    except NameError:
        print('No B')
    try: 
        del(creeping_C)
    except NameError:
        print('No C')
Creepmeter_dataframe.drop_duplicates('File_code',inplace=True)
Creepmeter_dataframe.reset_index(inplace=True)
Creepmeter_dataframe.drop(['index'], axis=1,inplace=True)
Creepmeter_dataframe.to_csv('../../../Data/creepmeter_lookup.csv')
print(Creepmeter_dataframe)

xmro


FileNotFoundError: ../../../Data/Central_San_Andreas_Fault/Raw/xmro_merge-2.jl not found.

In [4]:
creeping

,Time,Tm,Creep
0,2004-09-29 02:19:59.999981,2004-09-29 02:19:59.999981,-86.572
1,2004-09-29 02:30:00.000029,2004-09-29 02:30:00.000029,-86.625
2,2004-09-29 02:39:59.999990,2004-09-29 02:39:59.999990,-86.617
3,2004-09-29 02:50:00.000038,2004-09-29 02:50:00.000038,-86.612
4,2004-09-29 03:00:00.000000,2004-09-29 03:00:00.000000,-86.607
...,...,...,...
732264,2020-06-11 06:49:59.999981,2020-06-11 06:49:59.999981,-17.328
732265,2020-06-11 07:00:00.000029,2020-06-11 07:00:00.000029,-17.328
732266,2020-06-11 07:09:59.999990,2020-06-11 07:09:59.999990,-17.328
732267,2020-06-11 07:20:00.000038,2020-06-11 07:20:00.000038,-17.328


Import data and create .SAC files for Roger's creepmeters

In [30]:
for j in range(len(station)):
    creeping, tm_diff = rto.import_csv_roger(station[j],network)

    plt.figure()
    bins = np.arange(0,61,1)
    plt.hist(tm_diff,bins=bins)
    plt.show()


    bins_hist, hist_val = np.histogram(tm_diff,bins)
    boolarr = bins_hist > 50000
    periods = hist_val[1:][boolarr] -1
    print(periods)

    if len(periods) >= 2:
        print('warning, this needs thinking about majorly!')

        a = eval('rto.{k}_to_{g}(tm_diff)'.format(k = stringify.number_to_words(periods[0]), g = stringify.number_to_words(periods[1])))
        b = eval('rto.{k}_to_{g}(tm_diff)'.format(k = stringify.number_to_words(periods[1]), g = stringify.number_to_words(periods[0])))

        creeping_A = creeping.iloc[:a]
        creeping_B = creeping.iloc[a:b]
        creeping_C = creeping.iloc[b:]
    elif len(periods) == 1: 
        creeping_C = creeping.iloc[0:]
    

    Number_of_splits = []
    Sampling_rates_variable = []
    try:
        sampling_A = round((creeping_A.Tm.iloc[2] - creeping_A.Tm.iloc[1])/dt.timedelta(minutes=1))
        Sampling_rates_variable.append(sampling_A)
        Number_of_splits.append('A')
    except (IndexError, NameError):
        sampling_A = 0

    try:
        sampling_B = round((creeping_B.Tm.iloc[2] - creeping_B.Tm.iloc[1])/dt.timedelta(minutes=1))
        Sampling_rates_variable.append(sampling_B)
        Number_of_splits.append('B')
    except (IndexError, NameError):
        sampling_B = 0

    try:
        sampling_C = round((creeping_C.Tm.iloc[-1] - creeping_C.Tm.iloc[-2])/dt.timedelta(minutes=1))
        Sampling_rates_variable.append(sampling_C)
        Number_of_splits.append('C')
    except (IndexError, NameError):
        sampling_C = 0

    print(Sampling_rates_variable,Number_of_splits)
    print(creeping_C)
#

    for i in range(len(Number_of_splits)):
        print('interpolating creeping_{k}'.format(k=Number_of_splits[i]))
        upsampled = rto.interpolate(eval('creeping_{k}'.format(k=Number_of_splits[i])),Sampling_rates_variable[i])
        val1_okay = math.isnan(upsampled.Creep.iloc[0])
        if val1_okay == True:
            upsampled.reset_index(inplace=True)
            upsampled.drop([0],axis=0,inplace=True)
        tr = obspy.Trace(np.array(upsampled.Creep))
        st = obspy.Stream(tr)
        st[0].stats.network = network
        st[0].stats.station = station[j]
        st[0].stats.location = '00'
        st[0].stats.channel = 'slip'
        st[0].stats.starttime = pd.to_datetime(upsampled.Tm.iloc[0])
        st[0].stats.delta = 60*Sampling_rates_variable[i]
        #st[0].plot()
        instrument = pd.DataFrame({'Network':[st[0].stats.network],'Creepmeter_full_name':'please fill','Creepmeter_abbrv':'please fill','File_code':['{p}_{q}'.format(p=station[j],q=i)],'Start Time':[st[0].stats.starttime],\
                                   'End Time':[st[0].stats.endtime],'Sampling rate, Hz':[st[0].stats.sampling_rate],\
                                    'Sampling rate, mins':(1/st[0].stats.sampling_rate)/60,'Latitude':['please fill'],'Longitude':['please fill']})
        
        Creepmeter_dataframe = pd.concat([Creepmeter_dataframe,instrument])
        st.write(path +'/{p}_{q}.SAC'.format(p=station[j],q=i),format='SAC')
    try:
        del(creeping_A)
    except NameError:
        print('No A')
    try:
        del(creeping_B)
    except NameError:
        print('No B')
    try: 
        del(creeping_C)
    except NameError:
        print('No C')

#Creepmeter_dataframe.drop_duplicates('File_code',inplace=True)
Creepmeter_dataframe.reset_index(inplace=True)
Creepmeter_dataframe.drop(['index'], axis=1,inplace=True)
Creepmeter_dataframe.to_csv('../../../Data/creepmeter_lookup.csv')
print(Creepmeter_dataframe)

FileNotFoundError: [Errno 2] No such file or directory: '../../../Data/East_Anatolian_Fault/CSV/Gok.csv'

1

In [8]:
creeping_C

,Time,Tm,Creep
391188,2019-03-29 22:46:43,2019-03-29 22:46:43,-17.5654
391189,2019-03-29 23:00:18,2019-03-29 23:00:18,-17.5654
391190,2019-03-29 23:00:20,2019-03-29 23:00:20,-17.5654
391191,2019-03-29 23:00:22,2019-03-29 23:00:22,-17.5654
391192,2019-03-29 23:00:24,2019-03-29 23:00:24,-17.5654
...,...,...,...
2767610,2023-10-07 11:05:00,2023-10-07 11:05:00,3.3320
2767611,2023-10-07 11:06:00,2023-10-07 11:06:00,3.3320
2767612,2023-10-07 11:07:00,2023-10-07 11:07:00,3.3320
2767613,2023-10-07 11:08:00,2023-10-07 11:08:00,3.3300


Adding info to creepmeters database

In [18]:
Creepmeter_dataframe = pd.read_csv('../../../Data/creepmeter_lookup.csv',index_col=0)
Creepmeter_dataframe

,Network,Creepmeter_full_name,Creepmeter_abbrv,File_code,Start Time,End Time,"Sampling rate, Hz","Sampling rate, mins",Latitude,Longitude
0,Calaveras_Fault,Shore Rd #1,XSH1,xsh_0,2020-10-15T17:02:00.000000Z,2023-03-30T05:01:00.000000Z,0.016667,1,36.9430,-121.4450
1,Hayward_Fault,Fremont Winery,CFW1,cfw_0,1993-12-15T01:00:00.000000Z,1999-07-02T00:00:00.000000Z,0.000278,60,37.5320,-121.9520
2,Hayward_Fault,Fremont Winery,CFW1,cfw_1,1999-07-03T20:00:00.000000Z,2020-01-07T19:10:00.000000Z,0.001667,10,37.5320,-121.9520
3,Hayward_Fault,Fremont Winery,CFW1,cfw_2,2020-01-07T19:25:00.000000Z,2024-02-05T14:17:00.000000Z,0.016667,1,37.5320,-121.9520
4,Hayward_Fault,Hayward Palisades St #2,CHP1,chp_0,1994-04-02T01:20:00.000000Z,2020-01-07T22:10:00.000000Z,0.001667,10,37.6630,-122.0740
5,Hayward_Fault,Hayward Palisades St #3,CHP1,chp_1,2020-01-08T01:01:00.000000Z,2024-02-05T14:09:00.000000Z,0.016667,1,37.6630,-122.0740
6,Hayward_Fault,Oakland Zoo,COZ1,coz_0,1996-11-06T09:00:00.000000Z,2006-08-30T20:00:00.000000Z,0.000278,60,37.7530,-122.1500
7,Hayward_Fault,Oakland Zoo,COZ1,coz_1,2006-08-30T21:00:00.000000Z,2020-01-07T17:10:00.000000Z,0.001667,10,37.7530,-122.1500
8,Hayward_Fault,Oakland Zoo,COZ1,coz_2,2020-01-07T17:24:00.000000Z,2024-02-05T14:07:00.000000Z,0.016667,1,37.7530,-122.1500
9,Hayward_Fault,Pt Pinole,CPP1,cpp_0,1995-08-25T20:00:00.000000Z,2020-01-20T23:40:00.000000Z,0.001667,10,37.9900,-122.3560


Creating CSV areas

In [8]:
test_2 = pd.read_csv("../../../Data/East_Anatolian_Fault/Raw/Sivrice2020_3.txt")

In [9]:
test_2.rename(columns={'SivriceDay':'Date','Sivricemm':'Slip'},inplace=True)
test_2.drop(['SivricdegC'], axis=1,inplace=True)
test_2.to_csv('../../../Data/East_Anatolian_Fault/CSV/sri.csv')

In [ ]:
test_2 = pd.read_csv("../../../Data/Southern_San_Andreas_Fault/Ross_Road_Imperial_to_2022.txt")

In [ ]:
test_2

In [ ]:
test_2.drop(['coRRC'],axis=1)

In [3]:
rr_23 = pd.read_csv('../../../Southern_CA_creepmeter_data/Ross_road_2022_2023.txt')
rr_23.rename(columns = {'SoCalDate':'Date','RR':'Slip'},inplace=True)
rr_to_22 = pd.read_csv('../../../Southern_CA_creepmeter_data/Ross_Road_Imperial_to_2022.txt')
rr_to_22.rename(columns={'coRRDAY':'Date','coRR':'Slip'},inplace=True)
rr_to_22.drop(['coRRC'], axis=1,inplace=True)

Ross_road=pd.concat([rr_to_22,rr_23],ignore_index=True)

In [5]:
Ross_road

,Date,Slip
0,08/08/2010 20:25:22,-37.7481
1,08/08/2010 20:55:22,-37.7481
2,08/08/2010 21:25:22,-37.7481
3,08/08/2010 21:55:22,-37.7481
4,08/08/2010 22:25:22,-37.7841
...,...,...
2767611,10/07/2023 11:05:00,3.3320
2767612,10/07/2023 11:06:00,3.3320
2767613,10/07/2023 11:07:00,3.3320
2767614,10/07/2023 11:08:00,3.3300


In [6]:
Ross_road.to_csv('../../../Data/Southern_San_Andreas_Fault/CSV/rr.csv')

In [ ]:
Ross_road

In [ ]:
sh_23 = pd.read_csv('../../../Southern_CA_creepmeter_data/Superstition_to_Oct_2023.txt')
sh_23.rename(columns={'newSUtime':'Date','newSUmm1':'Slip','newSUC':'TempC'},inplace=True)
sh_23.Date = pd.to_datetime(sh_23.Date)
sh_23.to_csv('../../../Data/Southern_San_Andreas_Fault/CSV/sup2.csv')

In [ ]:
sh_04_19 = pd.read_csv('../../../Southern_CA_creepmeter_data/Superstition-2004_2019.txt')
sh_04_19.rename(columns={'SHday':'Date','SHmm':'Slip','SHT':'TempC'},inplace=True)
sh_04_19.Date = pd.to_datetime(sh_04_19.Date)
plt.figure()
plt.plot(sh_04_19.Date,sh_04_19.Slip)
plt.show()
sh_04_19.to_csv('../../../Data/Southern_San_Andreas_Fault/CSV/sup1.csv')

In [ ]:
sh_combo  = pd.concat([sh_04_19,sh_23])

In [ ]:
sh_combo.drop_duplicates(subset='Date',inplace=True)
sh_combo.to_csv('../../../Data/Southern_San_Andreas_Fault/CSV/sup.csv')

In [ ]:
plt.figure()
plt.plot(sh_combo.Date,sh_combo.Slip)
plt.show()

In [ ]:
durmid = pd.read_csv('../../../Data/Southern_San_Andreas_Fault/Raw/Durmid Hill Creep 2004-2024.txt')

In [ ]:
durmid.rename(columns={'SoCalDate':'Date','Durmid':'Slip'},inplace=True)
durmid.Date = pd.to_datetime(durmid.Date)
durmid.to_csv('../../../Data/Southern_San_Andreas_Fault/CSV/dmh.csv')

In [15]:
#wsz = pd.read_csv("../../../Data/North_Anatolian_Fault/CSV/wsz.csv",names=['Date','Slip'])
wsz.to_csv("../../../Data/North_Anatolian_Fault/CSV/wsz.csv")

In [14]:
#wsz.reset_index(inplace=True)
#wsz.drop([0,1],inplace=True)
#wsz.drop(['level_0','index'],axis=1,inplace=True)
#wsz.reset_index(inplace=True)
wsz.drop(['index'],axis=1,inplace=True)
wsz

,Date,Slip
0,2017-12-04 09:08:27,3.69
1,2017-12-04 09:13:27,3.69
2,2017-12-04 09:18:27,3.69
3,2017-12-04 09:23:27,3.69
4,2017-12-04 09:28:27,3.69
...,...,...
2312042,2023-06-07 12:16:17,8.2322
2312043,2023-06-07 12:17:17,8.2322
2312044,2023-06-07 12:18:17,8.2322
2312045,2023-06-07 12:19:17,8.2537


In [4]:
inw = pd.read_csv("../../Data/North_Anatolian_Fault/RAW/Ismetpasa North Wall 2004-2023.txt")
inw

,northday,northamm
0,05/22/2014 00:00:00,-51.763900
1,05/22/2014 00:38:27,-51.763900
2,05/22/2014 01:08:27,-51.763900
3,05/22/2014 01:38:27,-51.763900
4,05/22/2014 02:08:27,-51.763900
...,...,...
2005109,09/16/2023 10:08:00,0.379905
2005110,09/16/2023 10:09:00,0.379905
2005111,09/16/2023 10:10:00,0.379905
2005112,09/16/2023 10:11:00,0.380205


In [ ]:
inw.rename(columns={'northday':'Date','northamm':'Slip'},inplace=True)

In [ ]:
inw.to_csv("../../../Data/North_Anatolian_Fault/CSV/inw.csv")

In [8]:
isw = pd.read_csv("../../Data/North_Anatolian_Fault/RAW/South_wall_longer.txt", delimiter='\t')
isw

,IsmetpasSouthDay,IsmetpasaSouthDextral,IsmetpasaSouth1
0,5/10/18 13:17:55,-26.6381,-21.8524
1,5/10/18 13:18:55,-26.6465,-21.8593
2,5/10/18 13:19:55,-26.6465,-21.8593
3,5/10/18 13:20:55,-26.6465,-21.8593
4,5/10/18 13:21:55,-26.6315,-21.8470
...,...,...,...
2521406,26/9/24 21:40:00,10.8030,8.8622
2521407,26/9/24 21:50:00,10.8030,8.8622
2521408,26/9/24 22:00:00,10.8030,8.8622
2521409,26/9/24 22:10:00,10.8030,8.8622


In [9]:
isw.rename(columns={'IsmetpasSouthDay':'Date','IsmetpasaSouthDextral':'Slip'},inplace=True)

In [10]:
isw.drop(['IsmetpasaSouth1'],axis=1,inplace=True)

In [12]:
isw.to_csv("../../Data/North_Anatolian_Fault/CSV/isw.csv")

In [14]:
cha = pd.read_csv("../../../Data/Chaman_Fault/RAW/Chaman creep event 2019 _ hourly data.txt")

In [17]:
cha.rename(columns={'chamanDay':'Date','mm':'Slip'},inplace=True)
cha.to_csv("../../../Data/Chaman_Fault/CSV/cha.csv")

In [11]:
fc = pd.read_csv("../../../Data/Central_San_Andreas_Fault/Raw/FoxCreekSouth to March 2024.txt")
fc.drop(['FoxSouthNormal','FoxSouthdegC'],inplace=True,axis=1)
fc.rename(columns = {'FoxSouthOblique':'Slip','FoxSouthDAY':'Date'},inplace=True)
fc.to_csv("../../../Data/Central_San_Andreas_Fault/CSV/fxc.csv")

In [23]:
PaN = pd.read_csv("../../Data/East_Anatolian_Fault/Raw/Palu_North.txt", delimiter='\t')
PaN

,northday,northC,northmm,northV
0,24/7/17 07:00:44,25.234,-11.1886,3.55
1,24/7/17 07:01:44,25.271,-11.1905,3.52
2,24/7/17 07:02:44,25.310,-11.1905,3.50
3,24/7/17 07:03:44,25.333,-11.1905,3.52
4,24/7/17 07:04:44,25.346,-11.1905,3.50
...,...,...,...,...
2024178,27/5/24 09:16:36,17.259,0.6337,NaN
2024179,27/5/24 09:17:36,17.266,0.6337,NaN
2024180,27/5/24 09:18:36,17.284,0.6420,NaN
2024181,27/5/24 09:19:36,17.333,0.6456,NaN


In [24]:
PaN.drop(['northV','northC'],inplace=True,axis=1)
PaN.rename(columns = {'northmm':'Slip','northday':'Date'},inplace=True)
PaN.to_csv('pan.csv')

In [26]:
PaS = pd.read_csv("../../Data/East_Anatolian_Fault/Raw/Palu_South.txt", delimiter='\t')
PaS

,southday,southC,Southmm,southv
0,24/7/17 07:00:44,25.234,0.2830,0.2830
1,24/7/17 07:01:44,25.271,0.2811,0.2811
2,24/7/17 07:02:44,25.310,0.2811,0.2811
3,24/7/17 07:03:44,25.333,0.2811,0.2811
4,24/7/17 07:04:44,25.346,0.2811,0.2811
...,...,...,...,...
1497863,22/2/22 06:41:59,8.070,NaN,NaN
1497864,22/2/22 06:42:59,8.071,NaN,NaN
1497865,22/2/22 06:43:59,8.071,NaN,NaN
1497866,22/2/22 06:44:59,8.070,NaN,NaN


In [27]:
PaS.drop(['southv','southC'],inplace=True,axis=1)
PaS.rename(columns = {'Southmm':'Slip','southday':'Date'},inplace=True)
PaS.to_csv('pas.csv')

In [3]:
west_sazlik = pd.read_csv("../../Data/North_Anatolian_Fault/CSV/Sazlik_file_test.csv")

In [12]:
west_sazlik['Time'] = pd.to_datetime(west_sazlik['Time'])
west_sazlik['diff'] = west_sazlik['2019_event_excel'] - west_sazlik['Igor_sazlik_excel']
west_sazlik

,Time,2019_event_excel,Igor_sazlik_excel,diff
0,2019-10-09 12:08:00,0.570615,-19.2906,19.861215
1,2019-10-09 12:09:00,0.570215,-19.2910,19.861215
2,2019-10-09 12:10:00,0.570215,-19.2910,19.861215
3,2019-10-09 12:11:00,0.570215,-19.2910,19.861215
4,2019-10-09 12:12:00,0.570215,-19.2910,19.861215
...,...,...,...,...
147136,2020-01-19 20:34:00,9.519710,-10.3519,19.871610
147137,2020-01-19 20:35:00,9.519410,-10.3522,19.871610
147138,2020-01-19 20:36:00,9.519410,-10.3522,19.871610
147139,2020-01-19 20:37:00,9.519710,-10.3519,19.871610


In [20]:
%matplotlib qt
fig = plt.figure()
ax1= plt.subplot(2,1,1)
plt.plot(west_sazlik['Time'],west_sazlik['2019_event_excel'])
plt.plot(west_sazlik['Time'],west_sazlik['Igor_sazlik_excel']+19.87)
ax2 = plt.subplot(2,1,2,sharex=ax1)
plt.plot(west_sazlik['Time'],west_sazlik['diff'])
fig.set_size_inches(15,5)
plt.show()

In [8]:
dfs = pd.read_excel('../../Data/North_Anatolian_Fault/Raw/Sazlik2017-2023.xlsx', sheet_name=[0,1,2,3,4])

In [6]:
all_sheets_df

,date time,sazlik west,degC,SazlikEast,WSazlikmm,WSazlikIR,EastSazlickmm,datetimeWSazlik,mm
0,2017-12-04 09:08:27,3.69,7.624,7.5627,NaN,NaN,NaN,NaT,NaN
1,2017-12-04 09:13:27,3.69,7.624,7.5383,NaN,NaN,NaN,NaT,NaN
2,2017-12-04 09:18:27,3.69,7.624,7.5388,NaN,NaN,NaN,NaT,NaN
3,2017-12-04 09:23:27,3.69,7.624,7.5377,NaN,NaN,NaN,NaT,NaN
4,2017-12-04 09:28:27,3.69,7.531,7.5370,NaN,NaN,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...
370419,NaT,NaN,17.623,NaN,NaN,NaN,NaN,2023-06-07 12:16:17,8.2322
370420,NaT,NaN,17.626,NaN,NaN,NaN,NaN,2023-06-07 12:17:17,8.2322
370421,NaT,NaN,17.629,NaN,NaN,NaN,NaN,2023-06-07 12:18:17,8.2322
370422,NaT,NaN,17.636,NaN,NaN,NaN,NaN,2023-06-07 12:19:17,8.2537


In [16]:
dfs[0].rename(columns = {'sazlik west':'Slip W','date time':'Date','SazlikEast':'Slip E'},inplace=True)
dfs[1].rename(columns = {'WSazlikmm':'Slip W','date time':'Date','EastSazlickmm':'Slip E'},inplace=True)
dfs[2].rename(columns = {'WSazlikmm':'Slip W','date time':'Date','EastSazlickmm':'Slip E'},inplace=True)
dfs[3].rename(columns = {'mm':'Slip W','datetimeWSazlik':'Date'},inplace=True)

In [17]:
all_sheets_df = pd.concat(dfs.values())

In [19]:
all_sheets_df.reset_index(inplace=True,drop=True)
all_sheets_df

,Date,Slip W,degC,Slip E,WSazlikIR
0,2017-12-04 09:08:27,3.6900,7.624,7.5627,NaN
1,2017-12-04 09:13:27,3.6900,7.624,7.5383,NaN
2,2017-12-04 09:18:27,3.6900,7.624,7.5388,NaN
3,2017-12-04 09:23:27,3.6900,7.624,7.5377,NaN
4,2017-12-04 09:28:27,3.6900,7.531,7.5370,NaN
...,...,...,...,...,...
2312042,2023-06-07 12:16:17,8.2322,17.623,NaN,NaN
2312043,2023-06-07 12:17:17,8.2322,17.626,NaN,NaN
2312044,2023-06-07 12:18:17,8.2322,17.629,NaN,NaN
2312045,2023-06-07 12:19:17,8.2537,17.636,NaN,NaN


In [23]:
west_sazlik = pd.DataFrame({'Date':all_sheets_df['Date'],'Slip':all_sheets_df['Slip W']})
west_sazlik.dropna(inplace=True)
west_sazlik.reset_index(inplace=True,drop=True)
west_sazlik.to_csv("../../Data/North_Anatolian_Fault/CSV/wsz.csv")

In [24]:
east_sazlik = pd.DataFrame({'Date':all_sheets_df['Date'],'Slip':all_sheets_df['Slip E']})
east_sazlik.dropna(inplace=True)
east_sazlik.reset_index(inplace=True,drop=True)
east_sazlik.to_csv("../../Data/North_Anatolian_Fault/CSV/esz.csv")

In [25]:
west_sazlik = pd.DataFrame({'Date':all_sheets_df['Date'],'IR':all_sheets_df['WSazlikIR']})
west_sazlik.dropna(inplace=True)
west_sazlik.reset_index(inplace=True,drop=True)
west_sazlik.to_csv("../../Data/North_Anatolian_Fault/CSV/wsz_IR.csv")

In [26]:
west_sazlik = pd.DataFrame({'Date':all_sheets_df['Date'],'degC':all_sheets_df['degC']})
west_sazlik.dropna(inplace=True)
west_sazlik.reset_index(inplace=True,drop=True)
west_sazlik.to_csv("../../Data/North_Anatolian_Fault/CSV/Sazlik_Temp.csv")

In [29]:
df = pd.read_csv('../../Data/North_Anatolian_Fault/CSV/Hamamli_23AUG2019_20JAN2020.csv',header=None)

In [33]:
df.rename(columns = {0:'Date',1:'Slip'},inplace=True)
df.to_csv("../../Data/North_Anatolian_Fault/CSV/ham.csv")

In [36]:
df1 = pd.read_csv('../../Data/North_Anatolian_Fault/CSV/CherryPt1.csv')

In [43]:
df1['cherryday'] = pd.to_datetime(df1['cherryday'])
df1.rename(columns = {'cherryday':'Date','cherrymm':'Slip'},inplace=True)
df1

,Date,cherryC,Slip
0,2019-12-02 11:29:33,6.560,4.7534
1,2019-12-02 11:30:33,6.602,4.7463
2,2019-12-02 11:31:33,6.507,4.7489
3,2019-12-02 11:32:33,6.491,4.7430
4,2019-12-02 11:33:33,6.514,4.7432
...,...,...,...
342845,2019-08-10 13:34:33,13.373,0.8679
342846,2019-08-10 13:35:33,13.374,0.8679
342847,2019-08-10 13:36:33,13.374,0.8679
342848,2019-08-10 13:37:33,13.374,0.8677


In [41]:
df2 = pd.read_csv('../../Data/North_Anatolian_Fault/CSV/Cherry_09OCT2019_08FEB2020.csv',header=None)
df2.rename(columns = {0:'Date',1:'Slip'},inplace=True)
df2

,Date,Slip
0,10/09/2019 09:14:26,0.7370
1,10/09/2019 09:14:56,0.7370
2,10/09/2019 09:15:26,0.7351
3,10/09/2019 09:15:56,0.7324
4,10/09/2019 09:16:26,0.7349
...,...,...
350969,02/08/2020 05:58:56,7.1123
350970,02/08/2020 05:59:26,7.1130
350971,02/08/2020 05:59:56,7.1132
350972,02/08/2020 06:00:08,NaN


In [44]:
df_all = pd.concat([df1,df2])
df_all

,Date,cherryC,Slip
0,2019-12-02 11:29:33,6.560,4.7534
1,2019-12-02 11:30:33,6.602,4.7463
2,2019-12-02 11:31:33,6.507,4.7489
3,2019-12-02 11:32:33,6.491,4.7430
4,2019-12-02 11:33:33,6.514,4.7432
...,...,...,...
350969,02/08/2020 05:58:56,NaN,7.1123
350970,02/08/2020 05:59:26,NaN,7.1130
350971,02/08/2020 05:59:56,NaN,7.1132
350972,02/08/2020 06:00:08,NaN,NaN


In [45]:
df_cherry_T = pd.DataFrame({'Date':df_all['Date'],'degC':df_all['cherryC']})
df_cherry_T.dropna(inplace=True)
df_cherry_T.reset_index(inplace=True,drop=True)
df_cherry_T.to_csv("../../Data/North_Anatolian_Fault/CSV/Cherry_Temp.csv")

In [46]:
df_cherry = pd.DataFrame({'Date':df_all['Date'],'Slip':df_all['Slip']})
df_cherry.dropna(inplace=True)
df_cherry.reset_index(inplace=True,drop=True)
df_cherry.to_csv("../../Data/North_Anatolian_Fault/CSV/che.csv")

In [50]:
#inw = pd.read_csv('../../Data/North_Anatolian_Fault/CSV/North_Wall.csv')
inw.rename(columns = {'northday':'Date','northamm':'Slip'},inplace=True)
inw.to_csv("../../Data/North_Anatolian_Fault/CSV/inw.csv")

In [51]:
isw = pd.read_csv('../../Data/North_Anatolian_Fault/CSV/South_Wall.csv')
isw

,IsmetpasSouthDay,IsmetpasaSouthDextral,IsmetpasaSouth1
0,5/10/18 13:17:55,-26.6381,-21.8524
1,5/10/18 13:18:55,-26.6465,-21.8593
2,5/10/18 13:19:55,-26.6465,-21.8593
3,5/10/18 13:20:55,-26.6465,-21.8593
4,5/10/18 13:21:55,-26.6315,-21.8470
...,...,...,...
2521406,26/9/24 21:40:00,10.8030,8.8622
2521407,26/9/24 21:50:00,10.8030,8.8622
2521408,26/9/24 22:00:00,10.8030,8.8622
2521409,26/9/24 22:10:00,10.8030,8.8622


In [52]:
isw.rename(columns = {'IsmetpasSouthDay':'Date','IsmetpasaSouthDextral':'Slip'},inplace=True)
isw.to_csv("../../Data/North_Anatolian_Fault/CSV/isw.csv")

In [59]:
siv1 = pd.read_csv("../../Data/East_Anatolian_Fault/Raw/Sivrice2020_3.txt")
siv1.rename(columns={'SivriceDay':'Date','Sivricemm':'Slip','SivricdegC':'degC'},inplace=True)
siv1['Date'] = pd.to_datetime(siv1['Date'])
siv1

,Date,Slip,degC
0,2020-02-05 11:47:51,-120.5840,7.048
1,2020-02-05 11:48:01,-120.5900,7.100
2,2020-02-05 11:48:11,-120.4170,7.141
3,2020-02-05 11:48:21,-120.4110,6.851
4,2020-02-05 11:48:31,-120.4110,6.840
...,...,...,...
5104860,2023-12-02 11:15:00,5.0262,12.460
5104861,2023-12-02 11:16:00,5.0262,12.460
5104862,2023-12-02 11:17:00,5.0258,12.490
5104863,2023-12-02 11:18:00,5.0258,12.490


In [60]:
siv2 = pd.read_csv("../../Data/East_Anatolian_Fault/Raw/Sivrice_pt2.csv")
siv2.rename(columns={'SivDay':'Date','Sivmm':'Slip','SivC':'degC'},inplace=True)
siv2['Date'] = pd.to_datetime(siv2['Date'])
siv2

/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_60769/1377151943.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  siv2['Date'] = pd.to_datetime(siv2['Date'])


,Date,degC,Slip
0,2023-07-16 01:38:00,19.270,-3.2625
1,2023-07-16 01:39:00,19.250,-3.2629
2,2023-07-16 01:40:00,19.250,-3.2625
3,2023-07-16 01:41:00,19.250,-3.2625
4,2023-07-16 01:42:00,19.250,-3.2625
...,...,...,...
455292,2024-05-27 05:49:07,13.993,1.5115
455293,2024-05-27 05:50:07,14.153,1.5115
455294,2024-05-27 05:51:07,14.845,1.5115
455295,2024-05-27 05:51:24,NaN,NaN


In [63]:
siv = pd.concat([siv1,siv2],ignore_index=True)
siv.dropna(inplace=True)
siv.to_csv(("../../Data/East_Anatolian_Fault/CSV/siv.csv"))